In [8]:
import os, re, requests, tarfile
import random, shutil

word_map = {}
with open("words.txt", "r") as words:
    for line in words.readlines():
        lines_split = line.split("\t")
        word_map[lines_split[0]] = lines_split[1].replace("\n", "")

In [9]:
def extract_images(id, n):
    folder_path = f"image_net\\extracted_{id}"
    
    for _ in range(min(n, len(os.listdir(folder_path))-1)):
        folder_size = len(os.listdir(folder_path)) - 1
        rnd_idx = random.randint(0, folder_size)

        rnd_image = os.listdir(folder_path)[rnd_idx]

        if not os.path.exists(f"image_net\\{id}"):
            os.mkdir(f"image_net\\{id}")

        os.replace(f"{folder_path}\\{rnd_image}", f"image_net\\{id}\\{rnd_image}")
    
    shutil.rmtree(folder_path)

In [10]:
non_downloadable_ids = set()
with open("image_net\\non_downloadable_synsets.txt", "r") as non_downloadable_synsets:
    for line in non_downloadable_synsets.readlines():
        non_downloadable_ids.add(line.replace("\n", ""))
    
print(len(non_downloadable_ids))
print(len(word_map.keys()))

downloadable_ids = sorted(set(word_map.keys()).difference(non_downloadable_ids))
print(len(downloadable_ids))

7426
82115
74689


In [11]:
if not os.path.exists("image_net"):
    os.mkdir("image_net")

for id in downloadable_ids:
    if os.path.exists(f"image_net\\{id}"):
        continue

    print(f"\u001b[36m{list(word_map.keys()).index(id)}\u001b[0m ", end='')
    try:
        get_request = requests.get(f"https://image-net.org/data/winter21_whole/{id}.tar")
        if get_request.status_code == 200:
            print(f"Downloading {id} : {word_map[id]}")
            with open(f"image_net\\compressed_{id}.tar", "wb") as compressed_id:
                compressed_id.write(get_request.content)
            
            with tarfile.open(f"image_net\\compressed_{id}.tar") as compressed_tar:
                compressed_tar.extractall(f"image_net\\extracted_{id}")

            os.remove(f"image_net\\compressed_{id}.tar")

            # Extract 5 random images from the synset.
            extract_images(id, 5)
        else:
            print(f"\u001b[31mCan't download {id} : {word_map[id]}\u001b[0m")
            with open("image_net\\non_downloadable_synsets.txt", "a") as non_downloadable_synsets:
                non_downloadable_synsets.write(f"{id}\n")
    except Exception as e:
        print(e)


11 Downloading n00006024 : heterotroph
2185 Downloading n00443375 : dead-man's float, prone float
2189 Downloading n00443917 : gainer, full gainer
6815 Downloading n01333483 : vesiculovirus
7299 Downloading n01429172 : Ostariophysi, order Ostariophysi
8049 Downloading n01580772 : New World jay
8094 Downloading n01588002 : brown thrasher, brown thrush, Toxostoma rufums
8095 Can't download n01588172 : Xenicidae, family Xenicidae, Acanthisittidae, family Acanthisittidae
8096 Downloading n01588431 : New Zealand wren
8097 Can't download n01588589 : Xenicus, genus Xenicus
8098 Downloading n01588725 : rock wren, Xenicus gilviventris
8099 Can't download n01588858 : Acanthisitta, genus Acanthisitta
8100 Downloading n01588996 : rifleman bird, Acanthisitta chloris
8101 Can't download n01589125 : Certhiidae, family Certhiidae
8102 Downloading n01589286 : creeper, tree creeper
8103 Can't download n01589582 : Certhia, genus Certhia
8104 Downloading n01589718 : brown creeper, American creeper, Certhi

KeyboardInterrupt: 

In [ ]:
# delete excess files
for id in downloadable_ids:
    folder_path = f"image_net\\{id}"
    folder_size = len(os.listdir(folder_path))

    while folder_size > 5:
        rnd_idx = random.randint(0, folder_size-1)
        print(os.listdir(folder_path)[rnd_idx])
        # os.remove(os.listdir(folder_path)[rnd_idx])

In [22]:
with open("downloaded_index-word.txt", "w") as index_word_file:
    for id in os.listdir("image_net"):
        if id in word_map:
            index_word_file.write(f"{id}\t{word_map[id]}\n")

In [18]:
# SORT THE NON-DOWNLOADABLES
non_downloadable_ids = set()
with open("image_net\\non_downloadable_synsets.txt", "r") as non_downloadable_synsets:
    for line in non_downloadable_synsets.readlines():
        non_downloadable_ids.add(line.replace("\n", ""))

with open("image_net\\non_downloadable_synsets.txt", "w") as non_downloadable_synsets:
    non_downloadable_ids = sorted(non_downloadable_ids)
    for id in non_downloadable_ids:
        non_downloadable_synsets.write(f"{id}\n")